# Script para obtener datos de localidad de vecindarios de Ontario Canada

## Lectura de archivo con coordenadas, obtenido de la liga: http://cocl.us/Geospatial_data

In [2]:
import pandas as pd

# Lectura de archivo con coordenadas, obtenido de la liga: http://cocl.us/Geospatial_data
df_DS = pd.read_csv('Geospatial_Coordinates.csv', index_col=0)
df_DS.head(20)

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476
M1J,43.744734,-79.239476
M1K,43.727929,-79.262029
M1L,43.711112,-79.284577
M1M,43.716316,-79.239476


## Lectura del dataframe con los datos de vecindarios
## Lectura de la tabla usando read_html de pandas

In [3]:
# Lectura del dataframe con los datos de vecindarios
# Lectura de la tabla usando read_html de pandas
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

# Recorrido de cada finla del dataframe de lectura para generar filas por vecindario
df_final = pd.DataFrame()
for x in range(0,20):
    df_temp = df.iloc[x].str.extractall(r'(M\d\w)([^\(\(]*)(?:\((.*)\))?')
    df_final = pd.concat([df_final, df_temp])

#Limpieza de vlores nulos, ajuste de indice, eliminacion de columnas innecesarias y cambio de titulos
df_final.dropna(inplace=True)
df_final.reset_index(inplace=True)
df_final.drop(['level_0','match'], axis=1, inplace=True)
df_final.rename(columns={0: 'PostalCode', 1: 'Borough', 2: 'Neighborhood'}, inplace=True)
df_final.replace(' /',',', regex=True, inplace=True)

In [4]:
df_final.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


## Agregar columnas de ubicacion a dataframe original

In [5]:
# Agregar columnas de ubicacion a dataframe original
df_final['Latitude'] = df_final.PostalCode.map(df_DS['Latitude'].to_dict())
df_final['Longitude'] = df_final.PostalCode.map(df_DS['Longitude'].to_dict())

In [6]:
df_final.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [7]:
df_final

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


### Genera nuevo renglones para vecindarios en misma ciudad separados por coma

In [8]:
df_final['Neighborhood'] = df_final['Neighborhood'].str.split('[,]')
df_final = df_final.explode('Neighborhood').reset_index(drop=True)
cols = list(df_final.columns)
df_final = df_final[cols]

df_final

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M6A,North York,Lawrence Manor,43.718518,-79.464763
...,...,...,...,...,...
211,M8Z,Etobicoke,Mimico NW,43.628841,-79.520999
212,M8Z,Etobicoke,The Queensway West,43.628841,-79.520999
213,M8Z,Etobicoke,South of Bloor,43.628841,-79.520999
214,M8Z,Etobicoke,Kingsway Park South West,43.628841,-79.520999


## Conteo de numero de ciudades y vecindarios

In [9]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_final['Borough'].unique()),
        df_final.shape[0]
    )
)

The dataframe has 15 boroughs and 216 neighborhoods.


## Agrupado de vecindarios por nombre de vecindario y conteo

In [10]:
df_final.groupby('Borough')['PostalCode'].count()

Borough
Central Toronto                                                 16
Downtown Toronto                                                36
Downtown TorontoStn A PO Boxes25 The Esplanade                   1
East Toronto                                                     6
East TorontoBusiness reply mail Processing Centre969 Eastern     1
East York                                                        5
East YorkEast Toronto                                            1
Etobicoke                                                       44
EtobicokeNorthwest                                               9
MississaugaCanada Post Gateway Processing Centre                 1
North York                                                      36
Queen's Park                                                     1
Scarborough                                                     38
West Toronto                                                    13
York                                                  

In [11]:
df_final

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M6A,North York,Lawrence Manor,43.718518,-79.464763
...,...,...,...,...,...
211,M8Z,Etobicoke,Mimico NW,43.628841,-79.520999
212,M8Z,Etobicoke,The Queensway West,43.628841,-79.520999
213,M8Z,Etobicoke,South of Bloor,43.628841,-79.520999
214,M8Z,Etobicoke,Kingsway Park South West,43.628841,-79.520999


## Importacion de libreria para obtener latitud y longitud
### Se procede a obtener coordenadas de Ontario Canada

In [12]:
#conda install -c conda-forge geopy --yes # retirar el comentario de esta línea si no ha completado el laboratorio de la API de FourSquare
#pip install geopy
from geopy.geocoders import Nominatim # convertir una dirección en valores de latitud y longitud

In [13]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Ontario CA are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Ontario CA are 43.6534817, -79.3839347.


### Se genera mapa de Ontario, CA, con los vecindarios superpuestos

In [14]:
#!conda install -c conda-forge folium=0.5.0 --yes # retirar el comentario de esta línea si no ha completado el laboratorio de la API de FourSquare
import folium # librería para graficar mapas 

In [15]:
# crear un mapa de Nueva York utilizando los valores de latitud y longitud
map_ontario = folium.Map(location=[latitude, longitude], zoom_start=10)

# añadir marcadores al mapa
for lat, lng, borough, neighborhood in zip(df_final['Latitude'], df_final['Longitude'], df_final['Borough'], df_final['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ontario)  
    
map_ontario

### Trabajaremos con los datos del municipio de Central Toronto
Considerando que es el cuenta con una cantidad media aproximada de vecindarios

In [20]:
central_toronto_data = df_final[df_final['Borough'] == 'Central Toronto'].reset_index(drop=True)
central_toronto_data.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
5,M5R,Central Toronto,The Annex,43.672710,-79.405678
6,M5R,Central Toronto,North Midtown,43.672710,-79.405678
7,M5R,Central Toronto,Yorkville,43.672710,-79.405678
8,M4S,Central Toronto,Davisville,43.704324,-79.388790
9,M4T,Central Toronto,Moore Park,43.689574,-79.383160


### Obtenemos sus coordenadas y visualizamos vecindarios

In [17]:
address = 'Central Toronto, CA'

geolocator = Nominatim(user_agent="ct_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.66291795, -79.4088986364304.


In [18]:
# crear un mapa de Central Toronto usando los valores de latitud y longitud
map_central_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# añadir los marcadores al mapa
for lat, lng, label in zip(central_toronto_data['Latitude'], central_toronto_data['Longitude'], central_toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_central_toronto)  
    
map_central_toronto

### Obtener datos de Foursquare

In [29]:
CLIENT_ID = '---------' # su ID de Foursquare
CLIENT_SECRET = '---------' # Secreto de Foursquare
VERSION = '20223008' # versión de la API de Foursquare
LIMIT = 100 # Un valor límite para la API de Foursquare

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ---------
CLIENT_SECRET:---------


In [22]:
central_toronto_data.loc[0, 'Neighborhood']

'Lawrence Park'

In [26]:
central_toronto_data.loc[0, 'Latitude']
central_toronto_data.loc[0, 'Longitude']

-79.3887901

In [28]:
neighborhood_latitude = neighborhood_latitude = central_toronto_data.loc[0, 'Latitude'] # latitud del barrio 
neighborhood_longitude = neighborhood_latitude = central_toronto_data.loc[0, 'Latitude'] # latitud del barrio 
neighborhood_longitude = central_toronto_data.loc[0, 'Longitude'] # longitud del barrio

neighborhood_name = central_toronto_data.loc[0, 'Neighborhood'] # nombre del barrio

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


#### Obtengamos los 100 sitios en Central Toronto dentro de un radio de 500 metros.


In [ ]:
No se pudo continuar con la actividad ya que no esta aactualizado el script para trabajar con la version 3 de Foursquare

Se intentaron opciones pero se termino el credito gratuito para consultar despues de la primer consulta

In [ ]:
# escriba aquí su respuesta
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius


url = 'https://api.foursquare.com/v3/places/5a187743ccad6b307315e6fe/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


In [ ]:
import requests

url = "https://api.foursquare.com/v3/places/search"

headers = {
    "Accept": "application/json",
    "Authorization": "fsq3yhEVe5xcL5TibEefZixdaIfOtcowYZSY4PJyZS34b2c="
}

response = requests.get(url, headers=headers)